<a href="https://colab.research.google.com/github/evan-grinalds/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/Copy_of_LS_DS17_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [30]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler,RobustScaler
import scipy
import matplotlib.gridspec as gridspec
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split,KFold,cross_val_score,cross_validate
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import GridSearchCV
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import matplotlib.style as style

#### Continue to clean and explore your data.

In [32]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/evan-grinalds/Unit-2-Build-Tesla/master/tesla_model_s%20-%20model_s.csv')

print(df.shape)
df

(200, 7)


,year,car,battery,ludacris_mode,all_wheel_drive,mileage,price
0,2013,Model S,60,No,No,82851 mi.,27995
1,2018,Model S,100,No,Yes,5357 mi.,57992
2,2012,Model S,60,No,No,85478 mi.,24499
3,2017,Model S,100,No,Yes,32593 mi.,59980
4,2016,Model S,60,No,Yes,28418 mi.,49560
...,...,...,...,...,...,...,...
195,2014,Model S,60,No,No,25444 mi.,39590
196,2016,Model S,100,Yes,Yes,33719 mi.,67990
197,2020,Model S,100,No,No,1527 mi.,83900
198,2016,Model S,75,No,No,50600 mi.,40500


In [45]:
pd.options.display.float_format = '{:,.0f}'.format
df['price'].describe()

count      200
mean    47,691
std     15,380
min     24,499
25%     35,000
50%     44,585
75%     57,247
max     92,900
Name: price, dtype: float64

In [46]:
guess = df['price'].mean()

In [47]:
guess

47690.6

In [48]:
errors = guess - df['price']

In [49]:
errors

0      19,696
1     -10,301
2      23,192
3     -12,289
4      -1,869
        ...  
195     8,101
196   -20,299
197   -36,209
198     7,191
199     7,791
Name: price, Length: 200, dtype: float64

In [50]:
mean_absolute_error = errors.abs().mean()

In [51]:
print(f'If we just guessed every Tesla Model S sold for ${guess:,.0f},')
print(f'we would be off by ${mean_absolute_error:,.0f} on average.')

If we just guessed every Tesla Model S sold for $47,691,
we would be off by $12,357 on average.


In [ ]:
from sklearn.model_selection import train_test_split
train, test= train_test_split(df, test_size=0.20)

In [52]:
train.head(5)

,year,car,battery,ludacris_mode,all_wheel_drive,mileage,price
71,2014,Model S,60,No,No,82423 mi.,29942
65,2016,Model S,90,Yes,Yes,38958 mi.,64995
140,2013,Model S,60,No,No,28679 mi.,33999
152,2013,Model S,60,No,No,146487 mi.,25488
151,2015,Model S,85,No,No,84141 mi.,33347


In [53]:
test.head(5)

,year,car,battery,ludacris_mode,all_wheel_drive,mileage,price
156,2016,Model S,70,No,Yes,49306 mi.,38497
32,2017,Model S,100,No,Yes,32565 mi.,66000
27,2013,Model S,60,Yes,No,39300 mi.,31795
147,2015,Model S,70,No,Yes,38500 mi.,40995
95,2015,Model S,85,No,Yes,88599 mi.,28900


In [58]:
feature = ['battery']
target = ['price']
X_train = train[feature]
X_test = test[feature]
y_test = test[target]
y_train = train[target]

In [59]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import category_encoders as ce
import numpy as np
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import KNNImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipeline = make_pipeline(
    ce.BinaryEncoder(), 
    KNNImputer(), 
    StandardScaler(), 
    SelectKBest(f_regression), 
    Ridge()
)

param_distributions = {
    'knnimputer__n_neighbors': [3,4,5,6,7,8], 
    'selectkbest__k': range(1, len(X_train.columns)+1), 
    'ridge__alpha': [0.1, 1,8,9,10,15], 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=10, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 36 is smaller than n_iter=100. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0328s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0588s.) Setting batch_size=4.


Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1320s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 228 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 300 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 353 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:    3.6s finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [60]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)
pipeline = search.best_estimator_
y_pred = pipeline.predict(X_test)
predictions = np.expm1(y_pred)
predictions
mae = mean_absolute_error(Y_test, predictions)
print(f'Test MAE: ${mae:,.0f}')

Best hyperparameters {'selectkbest__k': 1, 'ridge__alpha': 0.1, 'knnimputer__n_neighbors': 3}
Cross-validation MAE 7705.662449467697


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in expm1
  """


TypeError: ignored